In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import sys
sys.path.append('../')

In [31]:
import numpy as np
import pandas as pd
import anndata
from rnasieve.preprocessing import model_from_raw_counts

In [14]:
# Example raw count prep
subset_bulk = anndata.read_h5ad('../example_data/muscle_subset_bulk.h5ad')
subset = anndata.read_h5ad('../example_data/muscle_subset.h5ad')

# Raw counts prep
counts_by_onto_class = {}
for sc in subset:
    cell_onto_class = sc.obs['cell_ontology_class'][0]
    if cell_onto_class not in counts_by_onto_class:
        counts_by_onto_class[cell_onto_class] = np.empty((sc.X.shape[0], 0), dtype=np.float32)
    counts_by_onto_class[cell_onto_class] = np.hstack((counts_by_onto_class[cell_onto_class], sc.X.reshape((-1, 1))))

# Bulk prep
G = subset_bulk.n_vars
bulk_by_age = {}
for bulk in subset_bulk:
    age = bulk.obs['Age'][0]
    if age not in bulk_by_age:
        bulk_by_age[age] = []
    bulk_by_age[age].append(bulk.X.reshape(-1, 1))

bulk_labels = []
psis = np.empty((G, 0), dtype=np.float32)
for age in sorted(bulk_by_age.keys()):
    bulks = bulk_by_age[age]
    for i in range(len(bulks)):
        bulk_labels.append("{} months, subject {}".format(age, i))
        psis = np.hstack((psis, bulks[i]))

/usr/local/lib/python3.7/site-packages/anndata/core/anndata.py:846: FutureWarning: In anndata v0.7+, arrays contained within an AnnData object will maintain their dimensionality. For example, prior to v0.7 `adata[0, 0].X` returned a scalar and `adata[0, :]` returned a 1d array, post v0.7 they will return two dimensional arrays. If you would like to get a one dimensional array from your AnnData object, consider using the `adata.obs_vector`, `adata.var_vector` methods or accessing the array directly.
  warn_flatten()


In [21]:
model, cleaned_psis = model_from_raw_counts(counts_by_onto_class, psis[:, :2])

In [126]:
model.predict(cleaned_psis)

,B cell,T cell,endothelial cell,macrophage,mesenchymal stem cell,skeletal muscle satellite cell
Bulk 0,0.027959,0.0,0.149951,0.243413,0.146380,0.432296
Bulk 1,0.037862,0.0,0.141346,0.286565,0.219226,0.315001


In [128]:
model.compute_marginal_confidence_intervals()

[[(-0.19694897509366274, 0.2528673112901627),
  (-0.12258931535119333, 0.12258931535119333),
  (-0.05741237671602631, 0.35731515691076676),
  (-0.05093143169279657, 0.5377580630709381),
  (-0.21012842470297996, 0.5028884721371275),
  (0.2750845035215425, 0.5895077012749277)],
 [(-0.1837377780281255, 0.25946123837141494),
  (-0.12176976093855874, 0.12176976093855874),
  (-0.0461883751451477, 0.32888065268067745),
  (0.028347078588426555, 0.5447835225807676),
  (-0.0970757410144995, 0.5355280496947445),
  (0.1542503724583583, 0.4757509798133833)]]

In [129]:
model.plot_proportions('bar').properties(title="Muscle Proportion Estimates")

alt.FacetChart(...)

In [130]:
model.plot_proportions('stacked').properties(title="Muscle Proportion Estimates")

alt.Chart(...)